# Verinin Yüklenmesi:

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("RecSysEDA").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/04 10:20:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
data_path = "/home/projects/bigdata-recommendation-engine/data/"

In [4]:
books_df = (
    spark.read
    .format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(f"{data_path}books.csv")
)

In [5]:
ratings_df = (
    spark.read
    .format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(f"{data_path}ratings.csv")
)

In [ ]:
books_df.cache()
ratings_df.cache()

DataFrame[book_id: int, user_id: int, rating: int]

In [7]:
books_df.count()

10000

In [8]:
ratings_df.count()

981756

# Veri Tiplerinin Doğrulanması:

## books.csv Veri Tipi Kontrolü

In [9]:
books_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- book_id: integer (nullable = true)
 |-- best_book_id: integer (nullable = true)
 |-- work_id: integer (nullable = true)
 |-- books_count: integer (nullable = true)
 |-- isbn: string (nullable = true)
 |-- isbn13: double (nullable = true)
 |-- authors: string (nullable = true)
 |-- original_publication_year: double (nullable = true)
 |-- original_title: string (nullable = true)
 |-- title: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- average_rating: string (nullable = true)
 |-- ratings_count: string (nullable = true)
 |-- work_ratings_count: string (nullable = true)
 |-- work_text_reviews_count: string (nullable = true)
 |-- ratings_1: double (nullable = true)
 |-- ratings_2: integer (nullable = true)
 |-- ratings_3: integer (nullable = true)
 |-- ratings_4: integer (nullable = true)
 |-- ratings_5: integer (nullable = true)
 |-- image_url: string (nullable = true)
 |-- small_image_url: string (nullable = true)


### Yorum:

**Kimlik alanları (id, book_id, best_book_id, work_id):** 
Spark integer olarak atamış. Bunlar sadece kimlik temsil eder, üzerinde matematiksel işlem yapılmaz. StringType kullanmak daha güvenli olur çünkü ileride farklı kaynaklardan birleştirme (join) yapılırken tip uyuşmazlıkları çıkabilir (örneğin MongoDB/Elastic id’leri string).

**isbn, isbn13:** 
isbn doğru şekilde string. isbn13 yanlış şekilde double. ISBN numarası numerik gibi görünür ama aslında kimliktir. Başında sıfır olabilir, çok büyük olabilir → bu yüzden StringType olmalı. 

**original_publication_year:**
double atanmış. Yıl bilgisi kesirli sayı olamaz → IntegerType olmalı. Hatta eksik/null değerler olabileceği için nullable IntegerType en uygunu. 

**average_rating:**
string atanmış. Ortalama puan sayısal değerdir (ör. 4.12) → DoubleType olmalı.

**ratings_count, work_ratings_count, work_text_reviews_count:** 
String olark atanmış. Bunlar sayı sayacı olduğu için IntegerType olmalı.(gerekirse LongType). 

**ratings_1, ratings_2, ratings_3, ratings_4, ratings_5:** 
Bunlar puan dağılımı için sayı sayacı. ratings_1 şu anda double, diğerleri integer. Tutarlılık için hepsi IntegerType (veya LongType) olmalı. 

**image_url, small_image_url:** string doğru. Bunlar URL metinleri olduğu için zaten string olmalı.

In [10]:
from pyspark.sql import functions as F

In [11]:
books_df = ( 
    books_df
    .withColumn("id", F.col("id").cast("string"))
    .withColumn("book_id", F.col("book_id").cast("string"))
    .withColumn("best_book_id", F.col("best_book_id").cast("string"))
    .withColumn("work_id", F.col("work_id").cast("string"))
    .withColumn("isbn13", F.col("isbn13").cast("string"))
    .withColumn("original_publication_year", F.col("original_publication_year").cast("int"))
    .withColumn("average_rating", F.col("average_rating").cast("double"))
    .withColumn("ratings_count", F.col("ratings_count").cast("int"))
    .withColumn("work_ratings_count", F.col("work_ratings_count").cast("int"))
    .withColumn("work_text_reviews_count", F.col("work_text_reviews_count").cast("int"))
    .withColumn("ratings_1", F.col("ratings_1").cast("int"))
)

In [12]:
books_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- best_book_id: string (nullable = true)
 |-- work_id: string (nullable = true)
 |-- books_count: integer (nullable = true)
 |-- isbn: string (nullable = true)
 |-- isbn13: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- original_publication_year: integer (nullable = true)
 |-- original_title: string (nullable = true)
 |-- title: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- average_rating: double (nullable = true)
 |-- ratings_count: integer (nullable = true)
 |-- work_ratings_count: integer (nullable = true)
 |-- work_text_reviews_count: integer (nullable = true)
 |-- ratings_1: integer (nullable = true)
 |-- ratings_2: integer (nullable = true)
 |-- ratings_3: integer (nullable = true)
 |-- ratings_4: integer (nullable = true)
 |-- ratings_5: integer (nullable = true)
 |-- image_url: string (nullable = true)
 |-- small_image_url: string (nullable = true)

## ratings.csv Veri Tipi Kontrolü

In [13]:
ratings_df.printSchema()

root
 |-- book_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- rating: integer (nullable = true)



### Yorum:
**book_id, user_id**:
ID alanları integer değil string tutulmalıdır.
**rating**:
Kullanıcılar tam sayı kullanarak puan verebiliyor, tam sayı  tanımı doğrudur.

In [14]:
ratings_df = (
    ratings_df.withColumn("book_id", F.col("book_id").cast("string"))
    .withColumn("user_id", F.col("user_id").cast("string"))
)

In [15]:
ratings_df.printSchema()

root
 |-- book_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)



# Veri Kalitesi Analizi (Null ve Duplicate Değerler)

### Sütunların Gereklilikleri:
**isbn, isbn13**: Daha önce de konuştuğumuz gibi, bunlar kitap endüstrisine özgü kimlik numaralarıdır ve kullanıcı tercihleriyle doğrudan bir ilişkisi yoktur.

**original_title**: Daha tutarlı olan "title" sütunu kullanılır.

**goodreads_book_id, best_book_id, work_id**: Bunlar da book_id gibi birer kimlik numarasıdır. ratings_csv ile join için book_id yeterlidir.

**average_rating, ratings_count, work_ratings_count, work_text_reviews_count, ratings_1, ratings_2, ratings_3, ratings_4, ratings_5**: Bunlar, kitabın popülerliği hakkında bilgi veren, önceden agregasyon yapılmış (özetlenmiş) sütunlardır. Biz, ratings.csv'deki ham ve daha detaylı veriyi kullanarak kendi analizlerimizi yapacağımız için, bu hazır özetleri kendimiz yaratacağız.

In [16]:
secilen_sutunlar = ["id", "book_id", "original_publication_year", "title", "language_code", "authors" ]

In [17]:
books_simple_df = books_df.select(secilen_sutunlar)
books_simple_df.show(5),


+---+-------+-------------------------+--------------------+-------------+--------------------+
| id|book_id|original_publication_year|               title|language_code|             authors|
+---+-------+-------------------------+--------------------+-------------+--------------------+
|  1|2767052|                     2008|The Hunger Games ...|          eng|     Suzanne Collins|
|  2|      3|                     1997|Harry Potter and ...|          eng|J.K. Rowling, Mar...|
|  3|  41865|                     2005|Twilight (Twiligh...|        en-US|     Stephenie Meyer|
|  4|   2657|                     1960|To Kill a Mocking...|          eng|          Harper Lee|
|  5|   4671|                     1925|    The Great Gatsby|          eng| F. Scott Fitzgerald|
+---+-------+-------------------------+--------------------+-------------+--------------------+
only showing top 5 rows



(None,)

## Null Değerlerin Bulunması

In [18]:
books_null_counts = books_simple_df.select(
    [ F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in books_simple_df.columns]
)

books_null_counts.show()

+---+-------+-------------------------+-----+-------------+-------+
| id|book_id|original_publication_year|title|language_code|authors|
+---+-------+-------------------------+-----+-------------+-------+
|  0|      0|                       21|    0|         1084|      0|
+---+-------+-------------------------+-----+-------------+-------+



In [19]:
ratings_null_counts = ratings_df.select(
    [ F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in ratings_df.columns ]
)
ratings_null_counts.show()

+-------+-------+------+
|book_id|user_id|rating|
+-------+-------+------+
|      0|      0|     0|
+-------+-------+------+



### Null Değerler Hakkında Yorum:
- ratings tablosu öneri sistemi için elzemdir, kullanıcıların tercihleri bu tablodadır. ratings tablosunda eksik veri olmadığı için bir null değerler için bir güncelleme gerekmez
- books tablosunda;
    - original_publication_year, okuyucuların hangi dönem kitapları sevdiği açısından önemlidir, null kayıtların veri içerindeki oranıyla %0,21, etkisi düşüktür. 0 ile güncellenir.
    - language_code, kullanıcıların okuduğu kitabın dili segmentasyon için çok önemlidir, "unknown" değeriyle güncellenir.
    

In [20]:
books_df.filter(F.col("original_publication_year") == 0).count()  # Orijinal verilerde 0 yılına kayıtlı kitap bulunmuyor. Yılı bilinmeyen kitaplar için ayrıştırıcı bir değer

0

In [21]:
books_simple_df = books_simple_df.withColumn("original_publication_year", F.when(F.col("original_publication_year").isNull(), 0).otherwise(F.col("original_publication_year")) )

In [22]:
year_grouped_books = (
    books_simple_df
    .groupBy("original_publication_year")
    .agg( F.count("book_id").alias("book_count") )
)
year_grouped_books.filter(F.col("original_publication_year") < 1000).orderBy(F.col("original_publication_year").desc()).show(50)

+-------------------------+----------+
|original_publication_year|book_count|
+-------------------------+----------+
|                      975|         1|
|                      800|         1|
|                      609|         1|
|                      397|         1|
|                      180|         1|
|                      119|         1|
|                        8|         1|
|                        0|        21|
|                      -17|         1|
|                     -300|         1|
|                     -330|         1|
|                     -335|         1|
|                     -350|         2|
|                     -380|         1|
|                     -385|         2|
|                     -390|         1|
|                     -400|         2|
|                     -401|         1|
|                     -411|         2|
|                     -430|         1|
|                     -431|         1|
|                     -440|         1|
|                     -44

In [23]:
books_simple_df = books_simple_df.withColumn( "language_code", F.when(F.col("language_code").isNull(), "unknown").otherwise(F.lower(F.col("language_code"))) )

In [24]:
langGroup_books = (
    books_simple_df
    .groupBy("language_code")
    .agg(F.count("book_id").alias("book_count") )
)
langGroup_books.show(50)

+--------------------+----------+
|       language_code|book_count|
+--------------------+----------+
|                 fre|        25|
|                  en|         4|
|               en-gb|       257|
|                 rus|         1|
|                 ind|        21|
|             unknown|      1084|
|                 per|         7|
|                 nor|         3|
|                 pol|         6|
|               en-us|      2069|
|bloody jack (bloo...|         1|
|                 vie|         1|
|                 ara|        64|
|                 por|         6|
|                 swe|         1|
|                 mul|         1|
|                 eng|      6340|
|                 jpn|         7|
|                  nl|         1|
|                 dan|         3|
|                 fil|         2|
|                 tur|         1|
|                 rum|         1|
|               en-ca|        58|
| ""a man named da...|         1|
|                 ita|         2|
|             

### Dil Kodları Yorum:
**en, eng, en-US, en-CA** gibi bazı diller yrışmış. Bunları tek kümeye almak daha tutarlı olur, standardize edilir.

In [25]:
books_langs_cleaned_df = books_simple_df.withColumn( "language_code",
    F.when(F.col("language_code").isin("en","en-us", "en-gb", "en-ca"), "eng")
    .when(F.col("language_code").contains("bloody jack"), "eng")
    .when(F.col("language_code").contains("a man named"), "eng")
    .otherwise(F.col("language_code"))
)
langGroup_books2 = (
    books_langs_cleaned_df
    .groupBy("language_code")
    .agg(F.count("book_id").alias("book_count") )
)
langGroup_books2.show(50)

+-------------+----------+
|language_code|book_count|
+-------------+----------+
|          fre|        25|
|          rus|         1|
|          ind|        21|
|      unknown|      1084|
|          per|         7|
|          nor|         3|
|          pol|         6|
|          vie|         1|
|          ara|        64|
|          por|         6|
|          swe|         1|
|          mul|         1|
|          eng|      8730|
|          jpn|         7|
|           nl|         1|
|          dan|         3|
|          fil|         2|
|          tur|         1|
|          rum|         1|
|          ita|         2|
|          spa|        20|
|          ger|        13|
+-------------+----------+



# Tek Değişkeli Analiz

In [26]:
books_final_df = books_langs_cleaned_df
books_final_df.select("original_publication_year").describe().show()


+-------+-------------------------+
|summary|original_publication_year|
+-------+-------------------------+
|  count|                    10000|
|   mean|                1977.8255|
| stddev|       177.37988615440943|
|    min|                    -1750|
|    max|                     2017|
+-------+-------------------------+



10k original_publication_year kaydı var, yılların ortalaması 1977 yani yakın tarihte daha çok kitap kaydı var , standart sapma 177 yani veri setindeki çoğu kitap 1800'lü yıllardan günümüze kadar olan yıllarda çıkmış, min değer -1750 yani milattan önceki kitaplarda mevcut, max 2017 yani veri setindeki en güncel kitapların yılı 2017.

In [27]:
ratings_df.select("rating").describe().show()

+-------+------------------+
|summary|            rating|
+-------+------------------+
|  count|            981756|
|   mean|3.8565335989797873|
| stddev|0.9839408559620033|
|    min|                 1|
|    max|                 5|
+-------+------------------+



981756 oy verilmiş, oyların ortalaması 3.85 yani kullanıcılar platformadaki kitapları iyi seviyesine yakın puanlıyor, standart sapma 0.98 yani kullanıcı oylamalarının çoğunluğu 2.87-4.85 arasında normal bir dağılım var, min değer 1 ve max değer 5, kullanıcılar 1-5 puanlayabildiği için bu değerler doğru, 1'den küçük veya 5'ten büyük olan aykırı değerler mevcut değil. 

# Çok Değişkenli Analiz

In [28]:
avg_ratings = (
    ratings_df
    .groupBy("book_id")
    .agg(
        F.round(F.avg("rating"), 2).alias("avg_rating")
    )
)
avg_ratings.orderBy(F.col("avg_rating").desc()).show(5)


+-------+----------+
|book_id|avg_rating|
+-------+----------+
|   7947|      4.82|
|   5207|      4.78|
|   6920|      4.78|
|   9566|      4.78|
|   8946|      4.77|
+-------+----------+
only showing top 5 rows



In [29]:
books_with_avg_rats = (books_final_df.join(avg_ratings, on="book_id", how="inner"))
books_with_avg_rats.printSchema()

root
 |-- book_id: string (nullable = true)
 |-- id: string (nullable = true)
 |-- original_publication_year: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- avg_rating: double (nullable = true)



In [30]:
books_with_avg_rats.show(1, truncate=False, vertical=True)

-RECORD 0-----------------------------------------------
 book_id                   | 6194                       
 id                        | 5700                       
 original_publication_year | 1980                       
 title                     | Waiting for the Barbarians 
 language_code             | eng                        
 authors                   | J.M. Coetzee               
 avg_rating                | 3.72                       
only showing top 1 row

